In [8]:
from crewai.knowledge.source.base_knowledge_source import BaseKnowledgeSource
from typing import List, Dict, Any
from pydantic import Field
import requests

class WeatherKnowledgeSource(BaseKnowledgeSource):
    """
    A knowledge source that fetches weather information from a public API.
    """
    city: str = Field( description="The city for which to fetch weather data." )

    def load_content(self) -> Dict[Any, str]:
        print(f"Fetching weather for {self.city}...")

        # Hardcoded coordinates for demo purposes (San Francisco)
        endpoint = "https://api.open-meteo.com/v1/forecast"
        params = {
                "latitude": 37.77,
                "longitude": -122.42,
                "current_weather": True
            }

        response = requests.get(endpoint, params=params)
        response.raise_for_status()
        weather_data = response.json().get("current_weather", {})
        formatted = self.format_weather(weather_data)
        return {self.city: formatted}
    
    def format_weather(self, data: dict) -> str:
        if not data:
            return "No weather data available."

        return (
            f"Current weather in {self.city}:\n"
            f"- Temperature: {data.get('temperature')}°C\n"
            f"- Wind Speed: {data.get('windspeed')} km/h\n"
            f"- Weather Code: {data.get('weathercode')}\n"
            f"- Time: {data.get('time')}"
        )
    
    def add(self) -> None:
        content = self.load_content()
        for _, text in content.items():
            chunks = self._chunk_text(text)
            self.chunks.extend(chunks)

        self._save_documents()
    
    async def aadd(self) -> None:
        """Async version of add method."""
        self.add()
    
    def validate_content(self) -> bool:
        """Validate that the content can be loaded successfully."""
        try:
            content = self.load_content()
            return bool(content)
        except Exception:
            return False


In [9]:
from crewai import Agent, LLM, Crew, Process, Task

weather_knowldge = WeatherKnowledgeSource(city="San Francisco")
llm = LLM(model="gpt-4", temperature=0.0)

weather_agent = Agent(
    name="WeatherAgent",
    role="Weather Reporter",
    goal="Answer questions about the current weather forecast",
    backstory="""
        You are a weather reporting agent that provides accurate and up-to-date weather information based on data fetched from a public weather API. 
        """,
    llm=llm,
    knowledge_sources=[weather_knowldge],
    verbose=True
)   

task = Task(
    description="What is the current weather in San Francisco?",
    expected_output="A detailed report on the current weather conditions in San Francisco.",
    agent=weather_agent
)

crew = Crew(
    name="WeatherCrew",
    agents=[weather_agent],
    tasks=[task],
    process= Process.sequential,
    verbose=True
)

result = crew.kickoff()
print("Final Result:", result)


╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  WeatherCrew                                                                                                    │
│  ID:                                                                                                            │
│  a3ab3ecc-9a64-42db-b7ec-7f33a239bd54                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Fetching weather for San Francisco...


╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: What is the current weather in San Francisco?                                                            │
│  ID: 7e013e24-3b39-48e1-93c1-d7b645dd7dc7                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔍 Knowledge Retrieval ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Knowledge Retrieval Started                                                                                    │
│  Status: Retrieving...                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 📚 Knowledge Retrieved ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Search Query:                                                                                                  │
│  Detailed current weather conditions in San Francisco                                                           │
│  Knowledge Retrieved:                                                                                           │
│  Additional Information: Current weather in San Francisco:                                                      │
│  - Temperature: 8.8°C                                                                                           │
│  - Wind Speed: 7.4 km/h                                                                                         │
│  - Weather Code: 0                                                                                              │
│  - Time: 2026-01-15T07:00                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Weather Reporter                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The current weather in San Francisco is 8.8°C with a wind speed of 7.4 km/h. The weather code is 0, which      │
│  typically means clear skies. The time of this report is 2026-01-15T07:00. Please note that weather conditions  │
│  can change rapidly, so it's always a good idea to check the forecast before heading out.                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  What is the current weather in San Francisco?                                                                  │
│  Agent:                                                                                                         │
│  Weather Reporter                                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final Result: The current weather in San Francisco is 8.8°C with a wind speed of 7.4 km/h. The weather code is 0, which typically means clear skies. The time of this report is 2026-01-15T07:00. Please note that weather conditions can change rapidly, so it's always a good idea to check the forecast before heading out.


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  WeatherCrew                                                                                                    │
│  ID:                                                                                                            │
│  a3ab3ecc-9a64-42db-b7ec-7f33a239bd54                                                                           │
│  Final Output: The current weather in San Francisco is 8.8°C with a wind speed of 7.4 km/h. The weather code    │
│  is 0, which typically means clear skies. The time of this report is 2026-01-15T07:00. Please note that         │
│  weather conditions can change rapidly, so it's always a good idea to check the forecast before heading out.    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯